In [1]:
import warnings
warnings.filterwarnings(action='ignore',
                        category=DeprecationWarning,
                        module='stable_baselines')
warnings.filterwarnings(action='ignore',
                        category=UserWarning,
                        module='stable_baselines')
warnings.filterwarnings("ignore", category=FutureWarning, module='tensorflow')
warnings.filterwarnings("ignore", category=FutureWarning, module='tensorboard')
warnings.filterwarnings("ignore", category=UserWarning, module='gym')

import gym
import PortfolioAllocationGym
import numpy as np
from stable_baselines import A2C
from stable_baselines.common.policies import MlpLnLstmPolicy #, MlpPolicy, MlpLstmPolicy
from stable_baselines.common.evaluation import evaluate_policy
from stable_baselines.common.env_checker import check_env
from stable_baselines.bench import Monitor
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from datetime import datetime

In [3]:
observations = ['daily_returns', 'ema_50', 'ema_200', 'bb_bbm', 'bb_bbh', 'bb_bbl','bb_bbhi', 'bb_bbli', 'stoch', 'stoch_signal', 'macd','macd_signal', 'obv']
env_kwargs = {'filename':'sp500.csv',
    'date_from':'2008-01-01',
    'date_to':'2017-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5, # approx US Treasury Note return
    'sample_size':10,
    'random_sample':False,
    'observations' : observations,
    'save_info' : True,
    #'report_point' : 252,
    'reward_function':'benchmark'}

train_env = gym.make('PortfolioAllocation-v0', **env_kwargs)
train_env = Monitor(train_env, 'monitor')

True


In [8]:
check_env(train_env)

In [9]:
venv, obs = train_env.get_sb_env()

In [10]:
model_kwargs =   {
    'gamma': 0.98,
    'n_steps': 1,
    'lr_schedule': 'linear',
    'learning_rate': 0.0012271673895088168,
    'ent_coef': 0.07287684944232961,
    'vf_coef': 0.6406022095870604,
    'max_grad_norm': 0.3,
    'policy_kwargs' : dict (
        n_lstm=512,
        )
    }

a2c_model = A2C(policy = MlpLnLstmPolicy, tensorboard_log="tensorboard",env = venv)
                #**model_kwargs)
#a2c_model.get_parameter_list()

In [11]:
# Random Agent, before training
mean_reward, std_reward = evaluate_policy(a2c_model, venv, n_eval_episodes=5)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")


day: 2516                 reward: 12.681                 daily_returns: 31.021                 benchmark: 12.681                 index : 18.340                 sharpe: 3.100                 cum. rtns: 16.398                 portf val: 1,163,979.64
day: 2516                 reward: 16.832                 daily_returns: 35.172                 benchmark: 16.832                 index : 18.340                 sharpe: 3.842                 cum. rtns: 54.036                 portf val: 1,540,359.12
day: 2516                 reward: 17.996                 daily_returns: 36.336                 benchmark: 17.996                 index : 18.340                 sharpe: 3.806                 cum. rtns: 58.233                 portf val: 1,582,329.70
day: 2516                 reward: 18.599                 daily_returns: 36.939                 benchmark: 18.599                 index : 18.340                 sharpe: 4.299                 cum. rtns: 81.571                 portf val: 1,815,709.72
day: 251

In [12]:
total_timesteps = 10* (len(venv.venv.envs[0].data.date.unique()))

In [13]:
#total_timesteps = 500 * (len(venv.data.date.unique())-1)
trained_a2c_model= a2c_model.learn(total_timesteps=total_timesteps, #callback=tbc.TensorBoardCallback(),
                                   tb_log_name='A2C_lstm_bmk'+datetime.now().strftime("%H-%M"))


day: 2516                 reward: 4.428                 daily_returns: 22.768                 benchmark: 4.428                 index : 18.340                 sharpe: 2.094                 cum. rtns: 5.132                 portf val: 1,051,317.20
day: 2516                 reward: -8.384                 daily_returns: 9.956                 benchmark: -8.384                 index : 18.340                 sharpe: 3.128                 cum. rtns: 40.367                 portf val: 1,403,672.85
day: 2516                 reward: 34.988                 daily_returns: 53.328                 benchmark: 34.988                 index : 18.340                 sharpe: 4.324                 cum. rtns: 113.134                 portf val: 2,131,335.24
day: 2516                 reward: -54.792                 daily_returns: -36.452                 benchmark: -54.792                 index : 18.340                 sharpe: 3.214                 cum. rtns: 55.380                 portf val: 1,553,797.74
day: 25

In [15]:
trained_a2c_model.save('ltsm_500_300_collapse.zip')

In [ ]:
trained_a2c_model.env.venv.envs[0].

In [15]:
mean_reward, std_reward = evaluate_policy(trained_a2c_model, venv, n_eval_episodes=5)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
day: 2516                 reward: 0.000                 sharpe: 0.000                  cum. rtns: nan                 portf val: nan
mean_reward:nan +/- nan


In [ ]:
'''

eval_kwargs = {'filename':'sp500.csv',
    'date_from':'2018-01-01',
    'date_to':'2020-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5,
    'reward_function':'daily_returns'}

eval_env =  Monitor(gym.make('PortfolioAllocation-v0', **eval_kwargs))

mean_reward, std_reward = evaluate_policy(trained_a2c_model, eval_env, n_eval_episodes=10)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
'''

In [17]:
trade_day = train_env.data.loc[0, :]

In [19]:
np.isnan(train_env.data.any())

tic              False
date             False
open             False
low              False
high             False
close            False
adj_close        False
ema_50           False
ema_200          False
bb_bbm           False
bb_bbh           False
bb_bbl           False
bb_bbhi          False
bb_bbli          False
stoch            False
stoch_signal     False
macd             False
macd_signal      False
obv              False
daily_returns    False
dtype: bool

In [14]:
trade_day.date.unique()[0]

numpy.datetime64('2008-01-02T00:00:00.000000000')

In [15]:
df=pd.read_csv('sp500_returns.csv', sep=',', parse_dates=['Date'])
df.set_index('Date', inplace=True)

In [17]:
df.loc[trade_day.date.unique()[0]].daily_returns

-1.4437843160646446

In [5]:
x=train_env.env.data.loc[500, :]["adj_close","shares_os","market_cap"]

KeyError: ('adj_close', 'shares_os', 'market_cap')

In [7]:
train_env.observation_space

Box([-inf -inf -inf ... -inf -inf -inf], [inf inf inf ... inf inf inf], (7000,), float32)

In [19]:
import json

json.dump(trained_a2c_model.env.venv.envs[0].info,open('test.json','w'))

AttributeError: 'Env' object has no attribute 'info'

In [22]:
trained_a2c_model.env.venv.envs[0].weights_history[1321]


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])